In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


In [3]:
df = pd.read_csv("simulated_data.csv")
df.head()

In [13]:
df['timestamp'] = pd.to_datetime(df['timestamp'])

As someone who is still newer to Python, I wanted to show my thought process on how to tackle this aggregation problem. I was able to make my code more efficient by research the TimeDelta function instead of manually updating the minutes,hours, days etc.! I think it's important to reflect!

In [78]:

#def tenMinAgg(time):
#  year = time.year
#  month = time.month
#  day = time.day
#  hour = time.hour
#  minute = int(time.minute)


#  new_min = minute + 10

#  if new_min > 59:
#    minute = new_min - 60
#    hour += 1

#    if hour > 23:
#      hour = 0
#      day += 1
#
#      if day > 31:
#        day = 0
#        month += 1
#

#    new_time = pd.Timestamp(year=year, month=month, day=day, hour=hour, minute=minute)
#  else:
#    minute = new_min
#    new_time = pd.Timestamp(year=year, month=month, day=day, hour=hour, minute=minute)

#  return new_time


#first_time = df.iloc[0,0]
#agg = []

#for i in range(len(df['timestamp'])):
#  first_time_dt = pd.to_datetime(first_time)
#  last_time_dt = tenMinAgg(first_time_dt)
#   for df['timestamp'] > first_time_dt & df['timestamp'] < last_time_dt:
#      guest_estimate_agg = mean(df['guest_estimate'])
#      agg.append(guest_estimate_agg)




In [36]:
#def tenMinAgg(time):
#  new_time = time + pd.Timedelta(minutes = 10)
#  new_time_pd = pd.to_datetime(new_time)
#  return new_time_pd


#first_time = df['timestamp'].min()
#guest_estimate_agg = []


#for i in range(len(df['timestamp'])):
#  first_time_dt = pd.to_datetime(first_time)
#  last_time_dt = tenMinAgg(first_time_dt)

#  agg_df = df[(df['timestamp']>=first_time_dt) & (df['timestamp']< last_time_dt)]
#  guest_estimate_agg_calc  = agg_df['guest_estimate'].mean()
#  guest_estimate_agg.append(guest_estimate_agg_calc)
#  first_time = last_time_dt





In [63]:
#print(len(df['timestamp']))

10800


Okay, so this code above gives us the 10 min aggregations for the whole dataset, but we need to get this to be for each bar instead..... I think we need to group by the bar first, lets do that by bar id


In [37]:
#bar_groups = df.groupby('bar_id')

In [40]:
#bar_groups.get_group(565) sanity check

timestamp             name  user_id            bar  bar_id  \
10    2025-01-17 22:35:43     Caleb Carter      182  Fenway Tavern     565   
12    2025-01-17 22:19:37      Grace Lewis      412  Fenway Tavern     565   
17    2025-01-17 21:54:44     Sophie Clark      217  Fenway Tavern     565   
18    2025-01-17 21:51:27  Charlotte Green      492  Fenway Tavern     565   
21    2025-01-17 21:48:23    Olivia Cooper      342  Fenway Tavern     565   
...                   ...              ...      ...            ...     ...   
10753 2024-10-19 04:30:25  Isabella Hughes      524  Fenway Tavern     565   
10773 2024-10-19 02:48:32       Grace Hill       20  Fenway Tavern     565   
10784 2024-10-19 01:52:07    Ryan Peterson      169  Fenway Tavern     565   
10787 2024-10-19 01:28:23     Grace Wilson      164  Fenway Tavern     565   
10796 2024-10-19 00:04:23     Ethan Taylor      513  Fenway Tavern     565   

       flow_rate  guest_estimate  
10            13              26  
12            13              16  
17            13              64  
18            13              68  
21            13               2  
...          ...             ...  
10753         13               3  
10773         13              32  
10784         13              61  
10787         13              11  
10796         13              20  

[1100 rows x 7 columns]

HERE is the actual code used for the aggregation.

In [77]:

def tenMinAgg(time):
  new_time = time + pd.Timedelta(minutes = 10)
  new_time_pd = pd.to_datetime(new_time)
  return new_time_pd



bar_groups = df.groupby('bar_id')

individ_bar_agg = {}



for bar_id, bar_group in bar_groups:
  first_time = bar_group['timestamp'].min()
  guest_estimate_agg = []
  constant_flow_rate = bar_group['flow_rate'].iloc[0]

  for i in range(len(bar_group['timestamp'])):
    first_time_dt = pd.to_datetime(first_time)
    last_time_dt = tenMinAgg(first_time_dt)

    agg_df = bar_group[(bar_group['timestamp']>=first_time_dt) & (bar_group['timestamp']< last_time_dt)]
    guest_estimate_agg_calc  = agg_df['guest_estimate'].mean()
    estimate_minus_fr = guest_estimate_agg_calc - constant_flow_rate
    if estimate_minus_fr < 0: #doing this so we don't have negative people in line.
        estimate_minus_fr = 0
    else:
          estimate_minus_fr = estimate_minus_fr
    guest_estimate_agg.append({'interval_start': first_time_dt, 'mean_guest_estimate_10min': guest_estimate_agg_calc, 'flow_rate': constant_flow_rate, 'line_length': estimate_minus_fr})
    first_time = last_time_dt

  individ_bar_agg[bar_id] = guest_estimate_agg


In [76]:
print(individ_bar_agg[565]) #sanity check

[{'interval_start': Timestamp('2024-10-19 00:04:23'), 'mean_guest_estimate_10min': 20.0, 'flow_rate': 13, 'estimate_mins_fr': 7.0}, {'interval_start': Timestamp('2024-10-19 00:14:23'), 'mean_guest_estimate_10min': nan, 'flow_rate': 13, 'estimate_mins_fr': nan}, {'interval_start': Timestamp('2024-10-19 00:24:23'), 'mean_guest_estimate_10min': nan, 'flow_rate': 13, 'estimate_mins_fr': nan}, {'interval_start': Timestamp('2024-10-19 00:34:23'), 'mean_guest_estimate_10min': nan, 'flow_rate': 13, 'estimate_mins_fr': nan}, {'interval_start': Timestamp('2024-10-19 00:44:23'), 'mean_guest_estimate_10min': nan, 'flow_rate': 13, 'estimate_mins_fr': nan}, {'interval_start': Timestamp('2024-10-19 00:54:23'), 'mean_guest_estimate_10min': nan, 'flow_rate': 13, 'estimate_mins_fr': nan}, {'interval_start': Timestamp('2024-10-19 01:04:23'), 'mean_guest_estimate_10min': nan, 'flow_rate': 13, 'estimate_mins_fr': nan}, {'interval_start': Timestamp('2024-10-19 01:14:23'), 'mean_guest_estimate_10min': nan, '

In [70]:
print(individ_bar_agg) #sanity check

{563: [{'interval_start': Timestamp('2024-10-19 00:07:54'), 'mean_guest_estimate_10min': 15.0, 'flow_rate': 29, 'estimate_mins_fr': -14.0}, {'interval_start': Timestamp('2024-10-19 00:17:54'), 'mean_guest_estimate_10min': nan, 'flow_rate': 29, 'estimate_mins_fr': nan}, {'interval_start': Timestamp('2024-10-19 00:27:54'), 'mean_guest_estimate_10min': nan, 'flow_rate': 29, 'estimate_mins_fr': nan}, {'interval_start': Timestamp('2024-10-19 00:37:54'), 'mean_guest_estimate_10min': 20.0, 'flow_rate': 29, 'estimate_mins_fr': -9.0}, {'interval_start': Timestamp('2024-10-19 00:47:54'), 'mean_guest_estimate_10min': nan, 'flow_rate': 29, 'estimate_mins_fr': nan}, {'interval_start': Timestamp('2024-10-19 00:57:54'), 'mean_guest_estimate_10min': nan, 'flow_rate': 29, 'estimate_mins_fr': nan}, {'interval_start': Timestamp('2024-10-19 01:07:54'), 'mean_guest_estimate_10min': nan, 'flow_rate': 29, 'estimate_mins_fr': nan}, {'interval_start': Timestamp('2024-10-19 01:17:54'), 'mean_guest_estimate_10mi